# Step By Step Pengerjaan (Deadline 3 September 2024)
- Cocokin data byPlaceData.csv dengan scrapetable_wisata.xlsx
- Mengubah konten menjadi bahasa indonesia, dengan mengambil data dari byPlaceData.csv
- Data yang tidak ada di scrapetable_wisata.xlsx langsung di drop saja di byPlaceData.csv (mengurangi storage)

# Data Gathering

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# dataLamaPath = 'data/new-fix-data.csv'
dataBaruPath = 'data/scrapetable_wisata.xlsx'
dataByPlace = 'data/byPlaceData.csv'
dataFiltered = 'data/filtered_byPlaceData.csv'

# dfOld = pd.read_csv(dataLamaPath)
dfNew = pd.read_excel(dataBaruPath)
dfByPlace = pd.read_csv(dataByPlace)
dfFiltered = pd.read_csv(dataFiltered)

In [ ]:
dfNew.head()

In [ ]:
dfByPlace.head()

# Data Assessing

## Cek Info DF

In [ ]:
# ByPlace
print('By Place')
print(dfByPlace.info())

print()

# new
print('New')
print(dfNew.info())

## Cek Nilai Null

In [ ]:
# ByPlace
print('By Place')
print(dfByPlace.isna().sum())

print()

# new
print('New')
print(dfNew.isna().sum())

## Cek Nilai Duplikat

In [ ]:
# ByPlace
print('By Place')
print(dfByPlace.duplicated().sum())

print()

# new
print('New')
print(dfNew.duplicated().sum())

## Pemilihan Kolom Data Bu Melany

In [ ]:
dfNew.head()

In [ ]:
print(dfNew.latitude.sample(1))
print(dfNew.longitude.sample(1))

print(dfByPlace.coordinates.sample(1))
print(dfByPlace.address.sample(1).values)

In [ ]:
print(dfNew.timezone.unique())
print(dfNew.review_count.sample(1))
print(dfNew.place_id.sample(1))
print(dfNew.city.unique())
print(dfNew['validasi di jogja dan tetangga'].unique())

# Data Cleaning

In [ ]:
# Dataset 1: Menghapus kolom dengan banyak missing values
dfByPlace_cleaned = dfByPlace.drop(columns=['is_rating_updated', 'is_reviews_updated'])

# Dataset 2: Menghapus kolom dengan banyak missing values dan mengisi missing values lainnya
dfNew_cleaned = dfNew.drop(columns=['price_level'])

# Mengisi missing values pada kolom numerik dengan median
dfNew_cleaned['phone_number'].fillna('Unknown', inplace=True)
# dfNew_cleaned['review_count'].fillna(dfNew_cleaned['review_count'].median(), inplace=True) (Dipertimbangkan)
dfNew_cleaned['rating'].fillna(dfNew_cleaned['rating'].median(), inplace=True)

# Mengisi missing values pada kolom string dengan 'Unknown'
dfNew_cleaned['website'].fillna('Unknown', inplace=True)
columns_to_fill = ['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'state']
for column in columns_to_fill:
    dfNew_cleaned[column].fillna('Unknown', inplace=True)

# Memeriksa kembali missing values setelah pembersihan
missing_values_dfByPlace_cleaned = dfByPlace_cleaned.isnull().sum()
missing_values_dfNew_cleaned = dfNew_cleaned.isnull().sum()

missing_values_dfByPlace_cleaned, missing_values_dfNew_cleaned

## Menggabungkan Data dan Filtering

In [ ]:
# Menggabungkan data dari kedua dataset berdasarkan kolom address
# Menentukan keyword untuk filtering
keywords = ["Jawa Tengah", "Central Java", "Yogyakarta", "Special Region of Yogyakarta"]

# Filter Dataset 1
filtered_dfByPlace = dfByPlace_cleaned[dfByPlace_cleaned['address'].str.contains('|'.join(keywords), case=False, na=False)]

# Filter Dataset 2
filtered_dfNew = dfNew_cleaned[dfNew_cleaned['full_address'].str.contains('|'.join(keywords), case=False, na=False)]

# Menggabungkan kedua dataset yang telah difilter
combined_dataset = pd.concat([filtered_dfByPlace, filtered_dfNew], ignore_index=True)

# Menampilkan jumlah baris dan kolom dari dataset yang digabungkan
combined_shape = combined_dataset.shape
combined_shape

## Filtering Data

In [ ]:
# Convert the names in both datasets to lowercase for better matching
# dfByPlace['name'] = dfByPlace['name'].str.lower()
# dfNew['name'] = dfNew['name']

# Filter out rows in byPlaceData that do not have a matching name in scrapetable_wisata
filtered_data = dfByPlace[(dfByPlace['name'].str.lower()).isin((dfNew['name']).str.lower())]

# Now filter based on the address containing 'Jawa Tengah', 'Central Java', or 'Yogyakarta'
filtered_data = filtered_data[
    filtered_data['address'].str.contains('Jawa Tengah|Central Java|Yogyakarta', case=False)
]

# Display the first few rows of the filtered data to verify
filtered_data.head()

In [ ]:
# Save the filtered data to a new CSV file
filtered_data_path = 'data/filtered_byPlaceData.csv'
filtered_data.to_csv(filtered_data_path, index=False)

filtered_data.head()

## Ambil Data dari DF Baru
Pengambilan data yang mungkin masih merupakan wisata dari data baru, tapi tidak ada di data utama

In [ ]:
unpickedData = dfNew[~dfNew['name'].str.lower().isin(filtered_data['name'].str.lower())]

filteredCols = filtered_data.columns
unpickedCols = unpickedData.columns

### Get Data Fix

Data yang disini sudah hasil pengambilan dari data baru, yang tidak ada di data lama

In [ ]:
fixedDf = pd.read_csv('data/fixed_data.csv')
fixedDf.info()

In [ ]:
fixedDf.shape[0]

In [ ]:
filtered_data.shape[0]

### Cek Nilai Null

In [ ]:
fixedDf.isna().sum()

## Feature Selection

### Memindahkan Nilai dari Kolom ke Kolom

In [ ]:
# Fungsi drop kolom
def dropCol(df, lsCol):
    col1 = lsCol[0]
    col2 = lsCol[1]
    
    if col1 == 'latitude' and col2 == 'longitude':
        df.drop(columns=[col1, col2], inplace=True)
    else:
        df.drop(columns=[col2], inplace=True)
    
    return df

# Fungsi Pemindah Nilai
def moveValues(col1, col2, df):
    # Kondisi coordinates
    if col1 == 'latitude' and col2 == 'longitude':
        # Menggabungkan koordinat dan menyimpannya pada kolom 'coordinates'
        df['coordinates'] = df.apply(lambda row: f"{row[col1]},{row[col2]}" if pd.isna(row['coordinates']) else row['coordinates'], axis=1)
        
        # Menghapus kolom latitude dan longitude
        df = dropCol(df, [col1, col2])
    else:
        # Mengecek baris dengan NaN pada col1
        mask_nullCol1 = df[col1].isna()
        
        if mask_nullCol1.any():
            df.loc[mask_nullCol1, col1] = df.loc[mask_nullCol1, col2]
        else:
            print('Data tidak ada isinya')
        
        # Menghapus kolom col1 dan col2
        df = dropCol(df, [col1, col2])
    
    return df

In [ ]:
# Kolom Address dan Full Address
fixedDf = moveValues('address', 'full_address', fixedDf)
# Kolom latitude dan longitude
fixedDf = moveValues('latitude', 'longitude', fixedDf)
# Kolom latitude dan longitude
fixedDf = moveValues('reviews', 'review_count', fixedDf)

### Hapus Kolom Sisa yang Tidak Digunakan

In [ ]:
delCols = ['is_rating_updated', 'is_reviews_updated', 'website', 
           'business_id', 'phone_number', 'Unnamed: 0.1', 'Unnamed: 0', 'price_level']

fixedDf.drop(columns=delCols, inplace=True)

In [ ]:
fixedDf.info()

## Menangani Nilai Null

1. timezone (Faishal)
   
Diisi dengan nilai unique, yaitu Asia/Jakarta

In [ ]:
print('Nilai unique: ', fixedDf.timezone.unique())
fixedDf.timezone.fillna('Asia/Jakarta', axis=0, inplace=True)

2. types (Akhdan)

Diisi sesuai dengan bidangnya, dilihat dari nilai unique

In [ ]:
print('Nilai unique: ', fixedDf.types.unique())
# fixedDf.timezone.fillna('Asia/Jakarta', axis=0, inplace=True)

3. city (Faishal)
   
Diisi dengan menggunakan regex dari full address

4. workday_timing dan closed_on (Faishal)

Ambil dari yang hari-hari

5. reviews dan rating (Akhdan)
   
Manual aja ambil dari maps, kalau memungkinkan

6. reviewer_name, rating_review, dan review_text (Akhdan)

ambil dari maps juga, masing2 data 1 ajaa kalo yang null

7. validasi di jogja dan tetangga (Akhdan)

Langsung isi pake nilai terbanyak aja, keknya udah di jogja dan sekitarnya semua ini

In [ ]:
fixedDf['validasi di jogja dan tetangga'].unique()

# Exploratory Data Analysis (EDA)

In [ ]:
# Mengatur gaya visualisasi
sns.set(style="whitegrid")

# 1. Memeriksa missing values
missing_values_dfByPlace = dfByPlace.isnull().sum()
missing_values_dfNew = dfNew.isnull().sum()

# 2. Visualisasi distribusi rating
plt.figure(figsize=(14, 6))

# Dataset 1
plt.subplot(1, 2, 1)
sns.histplot(dfByPlace['rating'], bins=20, kde=True)
plt.title('Distribusi Rating - Dataset 1')
plt.xlabel('Rating')
plt.ylabel('Frekuensi')

# Dataset 2
plt.subplot(1, 2, 2)
sns.histplot(dfNew['rating'], bins=20, kde=True)
plt.title('Distribusi Rating - Dataset 2')
plt.xlabel('Rating')
plt.ylabel('Frekuensi')

plt.tight_layout()
plt.show()

# 3. Visualisasi distribusi jumlah review
plt.figure(figsize=(14, 6))

# Dataset 1
plt.subplot(1, 2, 1)
sns.histplot(dfByPlace['reviews'], bins=20, kde=True)
plt.title('Distribusi Jumlah Review - Dataset 1')
plt.xlabel('Jumlah Review')
plt.ylabel('Frekuensi')

# Dataset 2
plt.subplot(1, 2, 2)
sns.histplot(dfNew['review_count'], bins=20, kde=True)
plt.title('Distribusi Jumlah Review - Dataset 2')
plt.xlabel('Jumlah Review')
plt.ylabel('Frekuensi')

plt.tight_layout()
plt.show()

missing_values_dfByPlace, missing_values_dfNew